In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
words = open("../names.txt").read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = dict([(c, i) for i, c in enumerate(['.']+chars)])
itos = dict([(i, c) for c, i in stoi.items()])
vocab_size = len(stoi.keys())

In [ ]:
# Data prep
block_size = 3
def build_dataset(words, block_size=3):
    # Constructing the input matrix
    X, Y = [],[]
    
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print("".join([itos[i] for i in context]) + " ---> " + ch)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

In [ ]:
# Model init
emb_dim = 10
hidden_dim = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, emb_dim), generator=g)
# Change 2 - Reducing the range of W1 results in a reduction in the number of dead neurons.
#W1 = torch.randn((block_size * emb_dim, hidden_dim), generator=g) * 0.2
# Change 5 - Adding the bnbias term renders the `b1` matrix unnecessary and spurious. 
#b1 = torch.randn(hidden_dim, generator=g) * 0.01
# Change 3 - Using a gain function takes guesswork out of this.
W1 = torch.randn((block_size * emb_dim, hidden_dim), generator=g) * ((5/3) / (block_size * emb_dim)**0.5)
#b1 = torch.randn(hidden_dim, generator=g) * 0.01

# Change 1 - Reducing the range of initial weights and setting bias to zero helps with the initialization problem.
#W2 = torch.randn((hidden_dim, vocab_size), generator=g)
#b2 = torch.randn(vocab_size, generator=g)
W2 = torch.randn((hidden_dim, vocab_size), generator=g) * 0.01 # target is E[W2] = 1/27; 40 becomes 4
b2 = torch.randn(vocab_size, generator=g) * 0

# Change 4 - BatchNorm paper - mean normalizing the weights(gain or gamma) and bias(or beta) forces gaussian distribution.
bngain = torch.ones((1, hidden_dim))
bnbias = torch.zeros((1, hidden_dim))

# Change 5 - Setting a running mean and std makes predicting values easy. No recalculating the mean and std.
bnmean_running = torch.zeros((1, hidden_dim))
bnstd_running = torch.ones((1, hidden_dim))

parameters = [C, W1, W2, b2, bngain, bnbias]

for p in parameters:
    p.requires_grad = True

lossi = []
stepi = []

minibatch_size = 32

In [ ]:
# Running the network
epochs = 200000
#lr = 0.01
for epoch in range(epochs):
    ix = torch.randint(0, Xtr.shape[0], (minibatch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    # Forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    # Change 5
    hpreact = embcat @ W1 #+ b1
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)
    
    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update weights
    #lr = lrs[epoch]
    lr = 0.1 if epoch < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    stepi.append(epoch)
    lossi.append(loss.log10().item())

    if epoch % 10000 == 0:
        print(f'{epoch:7d}/{epochs:7d}: {loss.item():.4f}')

In [ ]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xtest, Ytest),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 # + b1
  hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

# Baseline

In [ ]:
plt.plot(stepi, lossi);

We want to get rid of the hockey stick like optimization of loss at the beginning.
More specifically, we want the `0/ 200000: 23.9692` loss at the first 10000 steps to be better.

In [ ]:
# What are the logits after the first epoch?
plt.hist(logits.view(-1).tolist());

Takes on very extreme values. We would expect this to be drawn from a uniform distribution.

In [ ]:
plt.imshow((h.abs() > 0.9999).tolist(), cmap="Grays")

The white blocks there show the neurons that aren't firing

# Change 1

# What can we do?
We want to see to it that the logits are drawn from a uniform distribution.
probs -> counts -> logits -> W2, b2, h -> W1, b1

So scaling down W2 and b2 for the first epoch should help scale this down.

In [ ]:
plt.hist((W2 * 0.01).view(-1).tolist());

In [ ]:
# This is what a uniform distribution looks like.
1.0/27.0

In [ ]:
# Change 1
plt.plot(stepi, lossi);

In [ ]:
plt.hist((W2).view(-1).tolist());

In [ ]:
plt.imshow((h.abs() > 0.9999).tolist(), cmap="Grays")

We still have the `h` problem and hence inactive neurons `tanh`.

In [ ]:
plt.hist(h.view(-1).tolist());

We have a high number of 1's and -1's that the `tanh` function squashes.

In [ ]:
plt.hist(hpreact.view(-1).tolist());

# Change 2
`hpreact` takes on a wide range of values causing many neurons to be inactivated.

hpreact -> W1, b1

So scaling down W1 and b1 should help.
We want it to be unit gaussian

In [ ]:
plt.plot(stepi, lossi);

In [ ]:
plt.hist(h.view(-1).tolist(), 50);

In [ ]:
plt.imshow((h.abs() > 0.99).tolist(), cmap="gray", interpolation="nearest")

In [ ]:
plt.hist(hpreact.view(-1).tolist());

# Change 3
Is there a better way to do this?

Kaiming He, scale by the `gain/sqrt(fan_in)` where `gain` is predetermined for each non-linearity
tanh = 5/3

In [ ]:
plt.plot(stepi, lossi);

In [ ]:
plt.hist(h.view(-1).tolist(), 50);

In [ ]:
plt.imshow((h.abs() > 0.99).tolist(), cmap="gray", interpolation="nearest")

# Change 4

`BatchNorm` - Batch normalization

Mean normalize the the entire `hpreact` to ensure unit normal gaussian distribution.
We then multiply this term by a gain and add a bias term.

In [ ]:
plt.plot(stepi, lossi);

In [ ]:
plt.imshow((h.abs() > 0.99).tolist(), cmap="gray", interpolation="nearest")